In [ ]:
import tensorflow as tf
import tensorflow_addons as tfa

In [ ]:
tf.config.list_logical_devices('GPU')

In [ ]:
def getLiuModel(f=8):
    return tf.keras.Sequential(
    [
        tf.keras.layers.Input((97,115,97,1)),
        tf.keras.layers.Conv3D(filters=f*4,
                               kernel_size=1,
                               padding='same',
                               strides=1,
                               activation=None,
                               dilation_rate=1),
        tfa.layers.InstanceNormalization(),
        tf.keras.layers.Activation('relu', name='relu'),
        tf.keras.layers.MaxPool3D(pool_size=3, strides=2),
        tf.keras.layers.Conv3D(filters=f*32,
                               kernel_size=3,
                               padding='valid',
                               strides=1,
                               activation=None,
                               dilation_rate=2),
        tfa.layers.InstanceNormalization(), 
        tf.keras.layers.Activation('relu', name='relu2'),
        tf.keras.layers.MaxPool3D(pool_size=3, strides=2),
        tf.keras.layers.Conv3D(filters=f*64,
                               kernel_size=5,
                               padding='same',
                               strides=1,
                               activation=None,
                               dilation_rate=2),
        tfa.layers.InstanceNormalization(), 
        tf.keras.layers.Activation('relu', name='relu3'),
        tf.keras.layers.MaxPool3D(pool_size=3, strides=2),
        tf.keras.layers.Conv3D(filters=f*64,
                               kernel_size=3,
                               padding='same',
                               strides=1,
                               activation=None,
                               dilation_rate=2),
        tfa.layers.InstanceNormalization(), 
        tf.keras.layers.Activation('relu', name='relu4'),
        tf.keras.layers.MaxPool3D(pool_size=5, strides=2, padding='same'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024),
        tf.keras.layers.Dense(3, activation='softmax')
    ])
getLiuModel().summary()

In [ ]:
tf.keras.layers.Conv3D?

In [ ]:
tf.keras.layers.MaxPool3D?

In [ ]:
import tensorflow as tf

param_split = '-'
layers_split = ';'


def _get_pool(pool_text, input_layer):
    params = pool_text.lower().split(param_split)
    assert len(params) == 4, f'{pool_text}'
    if params[0] == 'pmax3':
        return tf.keras.layers.MaxPool3D(pool_size=int(params[1]),
                                         strides=int(params[2]),
                                         padding=str(params[3]))(input_layer)
    elif params[0] == 'pmax2':
        return tf.keras.layers.MaxPool2D(pool_size=int(params[1]),
                                         strides=int(params[2]),
                                         padding=str(params[3]))(input_layer)
    elif params[0] == 'pavg3':
        return tf.keras.layers.AvgPool3D(pool_size=int(params[1]),
                                         strides=int(params[2]),
                                         padding=str(params[3]))(input_layer)
    elif params[0] == 'pavg2':
        return tf.keras.layers.AvgPool2D(pool_size=int(params[1]),
                                         strides=int(params[2]),
                                         padding=str(params[3]))(input_layer)
    else:
        raise Exception(f'Unknown string {pool_text}')


def _get_conv(conv_text, input_layer):
    params = conv_text.lower().split(param_split)
    act = str(params[5]) if str(params[5]) != 'none' else None
    assert len(params) == 6, f'{conv_text}'
    if params[0] == 'c3':
        return tf.keras.layers.Conv3D(filters=int(params[1]),
                                      kernel_size=int(params[2]),
                                      strides=int(params[3]),
                                      padding=str(params[4]),
                                      activation=act)(input_layer)
    elif params[0] == 'c2':
        return tf.keras.layers.Conv2D(filters=int(params[1]),
                                      kernel_size=int(params[2]),
                                      strides=int(params[3]),
                                      padding=str(params[4]),
                                      activation=act)(input_layer)
    elif params[0] == 'cb3':
        conv = tf.keras.layers.Conv3D(filters=int(params[1]),
                                      kernel_size=int(params[2]),
                                      strides=int(params[3]),
                                      padding=str(params[4]),
                                      activation=None)(input_layer)
        conv = tf.keras.layers.BatchNormalization()(conv)
        conv = tf.keras.layers.Activation(act)(conv)
        return conv
    elif params[0] == 'cb2':
        conv = tf.keras.layers.Conv2D(filters=int(params[1]),
                                      kernel_size=int(params[2]),
                                      strides=int(params[3]),
                                      padding=str(params[4]),
                                      activation=None)(input_layer)
        conv = tf.keras.layers.BatchNormalization()(conv)
        conv = tf.keras.layers.Activation(act)(conv)
        return conv
    else:
        raise Exception(f'Unknown string {conv_text}')


def _get_fc(fc_text, input_layer):
    params = fc_text.lower().split(param_split)
    assert len(params) == 3, f'{fc_text}'
    if params[0] == 'fc':
        act = str(params[2]) if str(params[2]) != 'none' else None
        return tf.keras.layers.Dense(units=int(params[1]),
                                     activation=act)(input_layer)
    else:
        raise Exception(f'Unknown string {fc_text}')


def _get_layer(layer_text, input_layer):
    if layer_text.lower().startswith('c'):
        return _get_conv(layer_text, input_layer)
    elif layer_text.lower().startswith('p'):
        return _get_pool(layer_text, input_layer)
    elif layer_text.lower().startswith('fc'):
        return _get_fc(layer_text, input_layer)
    elif layers_text.lower() == 'flat':
        return tf.keras.layers.Flatten()
    else:
        raise Exception(f'Unknown string {layer_text}')


def string_to_model(layers_text, input_shape, num_output=3):
    input_layer = tf.keras.layers.Input(input_shape)
    layers_text = layers_text.replace('\n', '')
    layers_text = layers_text.split(layers_split)
    layers = []
    last_layer = input_layer
    for layer_text in layers_text:
        last_layer = _get_layer(layer_text, last_layer)
        layers.append(last_layer)
    last_layer = tf.keras.layers.Flatten()(last_layer)
    output_layer = tf.keras.layers.Dense(num_output, activation='softmax')(last_layer)
    return tf.keras.Model(inputs=input_layer, outputs=output_layer)

In [5]:
input_layer = tf.keras.layers.Input((96,96,1))


model = string_to_model(text,(193,193,1) )
model.summary()

NameError: name 'tf' is not defined

In [21]:
from deep_mri.model_zoo import model_factory
text = 

model = model_factory('str_MyModel', layers_text=text,input_shape=(193,193,1))
model.summary()

Model: "MyModel"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        [(None, 193, 193, 1)]     0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 193, 193, 16)      32        
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 191, 191, 32)      4640      
_________________________________________________________________
max_pooling2d_69 (MaxPooling (None, 95, 95, 32)        0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 93, 93, 64)        18496     
_________________________________________________________________
max_pooling2d_70 (MaxPooling (None, 46, 46, 64)        0         
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 44, 44, 128)       7385

In [ ]:
tf.keras.layers.Dense?